In [ ]:
import numpy as npi
import cv2 as cv2

from PIL import Image
from utils import plot_cv2_images

%matplotlib inline

# Image stitching

Image stitching or photo stitching is the process of combining multiple photographic images with overlapping fields of view to produce a segmented panorama or high-resolution image. Nowadays it is one of the most successful applications in computer vision, and in fact, you find this functionality in basically every modern smartphone. In this notebook, we will implement a solution to combine multiple images into a panorama image. Let's begin by importing two images. We call them *left* and *right*.

In [ ]:
left = cv2.imread('../images/DSC02931.JPG')
right = cv2.imread('../images/DSC02932.JPG')

left = cv2.cvtColor(left, cv2.COLOR_BGR2RGB)
right = cv2.cvtColor(right, cv2.COLOR_BGR2RGB)

plot_cv2_images(left, right, hide_ticks=True)

Given a pair of images like the ones above, we want to stitch them to create a panoramic scene. It is important to note that both images need to share some common region. Moreover, our solution has to be robust even if the pictures have differences in one or more of the following aspects:
- Scaling
- Angle
- Spacial position
- Capturing devices

An initial approach could be to extract key points using an algorithm such as *Harris Corners*. Then, we could try to match the corresponding key points based on some measure of similarity like *Euclidean distance*. As we know, corners have one nice property: they are *invariant to rotation*. It means that, once we detect a corner, if we rotate an image, that corner will still be there.
However, what if we rotate then scale an image? In this situation, we would have a hard time because corners are not invariant to scale! 

To address this limitation, methods like **SIFT** use Difference of Gaussians (DoD). The idea is to apply DoD on differently scaled versions of the same image. It also uses the neighboring pixel information to find and refine key points and corresponding descriptors.

Now let's see how to implement SIFT with OpenCV. It is interesting to know that SIFT was originally patented, but the patent expired in 2020. Let's start by constructing a SIFT object. We can pass different parameters to it which are optional and they are well explained in the [documentation](https://docs.opencv.org/3.4/d7/d60/classcv_1_1SIFT.html).

In [ ]:
sift = cv2.SIFT_create()

## 1. Keypoint detection

We can use two methods to calculate a SIFT descriptor with OpenCV.

- The `sift.detect()` function finds the *keypoints* in the images. You can pass a mask if you want to search only a part of image. Each keypoint is a special structure which has many attributes like its $(x,y)$ coordinates, size of the meaningful neighbourhood, angle which specifies its orientation, response that specifies strength of keypoints etc. Next, you can call `sift.compute()` which computes the *descriptors* from the keypoints we have found. It returns a description containing the image changes like translation and rotation that allow us to match same or similar keypoints on different images.
- If you didn't find keypoints, you can directly find keypoints and descriptors in a single step with the function, `sift.detectAndCompute()`. It returns the *keypoints* and a *$128$-dimensional feature vector* for each key point.

In [ ]:
kp_left, des_left = sift.detectAndCompute(left, None)
kp_right, des_right = sift.detectAndCompute(right, None)

We can easily visualize found keypoints with OpenCV using the `drawKeypoints` function ([documentation](https://docs.opencv.org/3.4/d4/d5d/group__features2d__draw.html#gab958f8900dd10f14316521c149a60433)).

In [ ]:
keypoints_drawn_left = cv2.drawKeypoints(left, kp_left, None, color=(0, 0, 255))
keypoints_drawn_right = cv2.drawKeypoints(right, kp_right, None, color=(0, 0, 255))

plot_cv2_images(left, keypoints_drawn_left, right, keypoints_drawn_right, titles=['Left', 'Left keypoints', 'Right', 'Right keypoints'])

## 2. Feature matching

As we can see, we have a large number of features from both images. Now, we would like to compare the 2 sets of features and stick with the pairs that show more similarity. We can match the feature descriptors using the **Brute-Force (BF) matcher**. It takes the descriptor of one feature in the first set and matches it with all other features in the second set according to some **distance function**. For the BF matcher, first we have to create the BF matcher object using `BFMatcher()`. It takes two optional params. First one is `normType`, that specifies the distance measurement to be used. By default, it is `NORM_L2` that is good for SIFT, SURF etc (`NORM_L1` is also there). If you are using binary string based descriptors like ORB, BRIEF, BRISK etc, `NORM_HAMMING` should be used, which used Hamming distance as measurement. 

The `crossCheck` bool parameter indicates whether the two features have to match each other to be considered valid. In other words, for a pair of features $(f1, f2)$ to considered valid, $f1$ needs to match $f2$ and $f2$ has to match $f1$ as the closest match as well. 

Once we have a matcher object, two important methods are `BFMatcher.match()` and `BFMatcher.knnMatch()`. First one returns the best match. Second method returns $k$ best matches where $k$ is specified by the user. It may be useful when we need to do additional work on that.

The BF matcher can be very slow for large datasets, therefore it can be useful to apply faster matching algorithms like **FLANN**. FLANN stands for *Fast Library for Approximate Nearest Neighbors*. It contains a collection of algorithms optimized for fast nearest neighbor search in large datasets and for high dimensional features. In this notebook we will use the BF matcher, but you can read [this page](https://docs.opencv.org/3.4/dc/dc3/tutorial_py_matcher.html) to learn more about FLANN.

In [ ]:
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
matches = bf.match(des_left,des_right)

OpenCV provides the `drawMatches` function to visualize the matches ([documentation](https://docs.opencv.org/3.4/d4/d5d/group__features2d__draw.html#ga7d77c42960a2916217ddf202173f9ed2)).

In [ ]:
matches_drawn = cv2.drawMatches(left, kp_left, right, kp_right, matches, None, matchColor=(0,0,255), flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
plot_cv2_images(matches_drawn)

Viewing all matches can be of little use and a bit confusing. Because of this, we **sort** the best matches in ascending order of their distances so that the best matches (with low distance) come to the front. Then we draw only the **first 10 matches** (just for sake of visibility. You can increase it as you like).

In [ ]:
limit = 10
best = sorted(matches, key = lambda x:x.distance)[:limit]

In [ ]:
best_matches_drawn = cv2.drawMatches(left, kp_left, right, kp_right, best, None, matchColor=(0,0,255), flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
plot_cv2_images(best_matches_drawn)

## 3. Homography estimation using RANSAC and perspective warping

Until now we were able to find some feature points in the images and find the best matches among them. Notice that these steps are typically also used for *image retrieval* applications in which, given a query image, we need to return all the images that are more similar to the query image (a bit like Google images).

In our case, however, we want to transform the image on the right in order to stitch it with the one on the left. Nevertheless, the matcher algorithm will give us the best (more similar) set of features from both images. Now, we need to take these points and find the transformation matrix that will stitch the 2 images together based on their matching points. Such a transformation is called the **Homography matrix**. Briefly, the homography is a $3\times3$ matrix that can be used in many applications such as camera pose estimation, perspective correction, and image stitching. The Homography is a 2D transformation.

We will start by converting the best matches to coordinates on the left and right picture.

In [ ]:
left_pts = []
right_pts = []
for match in best:
    l_match = kp_left[match.queryIdx].pt
    r_match = kp_right[match.trainIdx].pt
    left_pts.append(l_match)
    right_pts.append(r_match)

Now we compute the transformation. Remind that the homography matrix is estimated with the **RANSAC algorithm**, which is an iterative algorithm to fit linear models and it's designed to be robust to outliers. We can use the `cv2.findHomography()` function ([documentation](https://docs.opencv.org/3.4/d9/d0c/group__calib3d.html#ga4abc2ece9fab9398f2e560d53c8c9780)).

In [ ]:
M, _ = cv2.findHomography(np.float32(right_pts), np.float32(left_pts))

Once we have the estimated homography, we need to warp one of the images to a common plane. Use the `cv2.warpPerspective()` function ([documentation](https://docs.opencv.org/4.x/da/d54/group__imgproc__transform.html#gaf73673a7e8e18ec6963e3774e6a94b87)).

In [ ]:
dim_x = left.shape[1] + right.shape[1]
dim_y = left.shape[0] + right.shape[0]
dim = (dim_x, dim_y)

warped = cv2.warpPerspective(right, M, dim)

In [ ]:
plot_cv2_images(warped)

Finally, we warp the *left* image to the *right* based on the homography.

In [ ]:
panorama = warped.copy()
# combine the two images
panorama[0:left.shape[0], 0:left.shape[1]] = left

# crop
h_crop = max(left.shape[0], right.shape[0])
w_crop = 2800
panorama = panorama[:h_crop, :w_crop]
plot_cv2_images(panorama)

As we see, there are a couple of artifacts in the result related to lighting conditions and edge effects at the image boundaries. Ideally, we can perform post-processing techniques to normalize the intensities like **histogram matching**. This would likely make the result look more realistic! If you are interested, you can take it as an extra exercise. You can read [this article](https://towardsdatascience.com/histogram-matching-ee3a67b4cbc1) to learn more about histogram matching.

## References

- [https://github.com/davidmasek/image_stitching](https://github.com/davidmasek/image_stitching)
- [https://towardsdatascience.com/image-panorama-stitching-with-opencv-2402bde6b46c](https://towardsdatascience.com/image-panorama-stitching-with-opencv-2402bde6b46c)